#Introduction & Motivation

Bulk RNA-seq analysis is a cornerstone of modern molecular biology, enabling the study of genome-wide transcriptional changes across biological conditions. Classical differential expression (DE) methods such as DESeq2 model read counts using a Negative Binomial distribution and test genes independently for association with experimental conditions. These approaches are statistically well-founded and have been highly successful in identifying differentially expressed genes (DEGs) across a wide range of studies.

However, classical DE analysis also has **inherent limitations**. Each gene is modeled independently, biological variation is represented using predefined and discrete condition labels (0 or 1 for the presence of a disease), and genes with low or sparse counts often yield unstable estimates or NA p-values, leading to their exclusion from downstream interpretation. Moreover, while DESeq2 is effective at hypothesis testing, it does not explicitly model shared structure across genes or attempt to learn a low-dimensional representation of the underlying biological state driving transcriptional variation.

Motivated by these considerations, the goal of this project is not to replace DESeq2, but to explore whether deep generative models can provide a complementary perspective on bulk RNA-seq data. In particular, I aim to investigate whether a probabilistic latent-variable model can (i) learn biologically meaningful structure in an unsupervised manner, (ii) model gene expression changes as smooth functions of latent biological states, and (iii) provide interpretable differential expression estimates even for genes that are challenging for classical statistical testing.

To this end, I focus on the publicly available **GEO dataset GSE60450**, which profiles bulk RNA-seq data from mouse mammary gland tissue under two physiological conditions: virgin and lactation (6 samples each). This dataset exhibits strong, well-characterized biological signal related to metabolic reprogramming and tissue remodeling during lactation, making it a suitable benchmark for both classical and generative analyses.

In this first notebook, I load the raw gene-level count matrix, construct a clean and consistent sample metadata table required for downstream analysis. The processed count matrix and metadata generated here will serve as the foundation for generative differential expression modeling in the subsequent notebook.

In [ ]:
import pandas as pd

Un-zipping the counts matrix downloaded from GEO database & reviewing it through Pandas...

In [ ]:
!gunzip GSE60450_Lactation-GenewiseCounts.txt.gz

In [ ]:
counts = pd.read_csv(
    "GSE60450_Lactation-GenewiseCounts.txt",
    sep="\t",
    index_col=0
)

print(counts.shape)
counts.head()

(27179, 13)


,Length,MCL1-DG_BC2CTUACXX_ACTTGA_L002_R1,MCL1-DH_BC2CTUACXX_CAGATC_L002_R1,MCL1-DI_BC2CTUACXX_ACAGTG_L002_R1,MCL1-DJ_BC2CTUACXX_CGATGT_L002_R1,MCL1-DK_BC2CTUACXX_TTAGGC_L002_R1,MCL1-DL_BC2CTUACXX_ATCACG_L002_R1,MCL1-LA_BC2CTUACXX_GATCAG_L001_R1,MCL1-LB_BC2CTUACXX_TGACCA_L001_R1,MCL1-LC_BC2CTUACXX_GCCAAT_L001_R1,MCL1-LD_BC2CTUACXX_GGCTAC_L001_R1,MCL1-LE_BC2CTUACXX_TAGCTT_L001_R1,MCL1-LF_BC2CTUACXX_CTTGTA_L001_R1
EntrezGeneID,,,,,,,,,,,,,
497097,3634,438,300,65,237,354,287,0,0,0,0,0,0
100503874,3259,1,0,1,1,0,4,0,0,0,0,0,0
100038431,1634,0,0,0,0,0,0,0,0,0,0,0,0
19888,9747,1,1,0,0,0,0,10,3,10,2,0,0
20671,3130,106,182,82,105,43,82,16,25,18,8,3,10


#Removal of the Gene Length Column

The original count matrix provided includes a column labeled Length, corresponding to the genomic length of each gene. While gene length is relevant for normalized expression measures such as RPKM or TPM, it is not used in count-based differential expression frameworks.

Both DESeq2 and the generative modelling approach adopted in this project explicitly model raw integer counts using a Negative Binomial distribution, with normalization handled via sample-specific size factors rather than gene length correction. Including the Length column in the count matrix would therefore introduce a non-count covariate that is incompatible with the assumed statistical model.

Accordingly, the Length column is removed to retain a clean gene-by-sample count matrix suitable for downstream probabilistic modeling.

In [ ]:
counts = counts.drop(columns=["Length"])

print(counts.shape)

(27179, 12)


#Creating the metadata for further analysis

A sample metadata table is constructed to encode experimental condition information for each RNA-seq sample. This metadata links each column of the count matrix to its corresponding biological condition (virgin or lactation) and will be used consistently across both classical and generative analyses to ensure proper alignment between expression data and sample-level annotations.

In [ ]:
metadata = pd.DataFrame(index=counts.columns)

metadata["condition"] = metadata.index.map(
    lambda x: "virgin" if "-DG_" in x or "-DH_" in x or "-DI_" in x or "-DJ_" in x or "-DK_" in x or "-DL_" in x
    else "lactation"
)

print(metadata)
print(metadata["condition"].value_counts())

                                   condition
MCL1-DG_BC2CTUACXX_ACTTGA_L002_R1     virgin
MCL1-DH_BC2CTUACXX_CAGATC_L002_R1     virgin
MCL1-DI_BC2CTUACXX_ACAGTG_L002_R1     virgin
MCL1-DJ_BC2CTUACXX_CGATGT_L002_R1     virgin
MCL1-DK_BC2CTUACXX_TTAGGC_L002_R1     virgin
MCL1-DL_BC2CTUACXX_ATCACG_L002_R1     virgin
MCL1-LA_BC2CTUACXX_GATCAG_L001_R1  lactation
MCL1-LB_BC2CTUACXX_TGACCA_L001_R1  lactation
MCL1-LC_BC2CTUACXX_GCCAAT_L001_R1  lactation
MCL1-LD_BC2CTUACXX_GGCTAC_L001_R1  lactation
MCL1-LE_BC2CTUACXX_TAGCTT_L001_R1  lactation
MCL1-LF_BC2CTUACXX_CTTGTA_L001_R1  lactation
condition
virgin       6
lactation    6
Name: count, dtype: int64


#Checking Library size consistency across samples

To assess potential technical variability, total read counts is computed for each sample. This step serves as a basic quality control check to ensure that library sizes are of comparable magnitude and that no sample exhibits extreme deviations that could disproportionately influence downstream modeling. Sample-specific normalization factors are applied later to account for residual differences in sequencing depth.

In [ ]:
counts.sum().sort_values()

,0
MCL1-DL_BC2CTUACXX_ATCACG_L002_R1,20015386
MCL1-LA_BC2CTUACXX_GATCAG_L001_R1,20392113
MCL1-DK_BC2CTUACXX_TTAGGC_L002_R1,21529331
MCL1-LB_BC2CTUACXX_TGACCA_L001_R1,21708152
MCL1-DH_BC2CTUACXX_CAGATC_L002_R1,21777891
MCL1-LD_BC2CTUACXX_GGCTAC_L001_R1,21988240
MCL1-LC_BC2CTUACXX_GCCAAT_L001_R1,22241607
MCL1-DJ_BC2CTUACXX_CGATGT_L002_R1,22665371
MCL1-DG_BC2CTUACXX_ACTTGA_L002_R1,23227641
MCL1-DI_BC2CTUACXX_ACAGTG_L002_R1,24100765


Sanity check on counts...

In [ ]:
assert (counts.values % 1 == 0).all()

Saving both files. I will use them in the next notebook...

In [ ]:
counts.to_csv("counts_matrix.tsv", sep="\t")
metadata.to_csv("metadata.tsv", sep="\t")